In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} torchvision

In [2]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c rdkit rdkit

In [1]:
from tensorboard_logger import configure, log_value
import tensorboard
import argparse
import os
import numpy as np
import math
import sys
import glob
#import utils import Logger


import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

### Part to use my own images

In [2]:
import glob
import random
import os
import numpy as np

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class ImageDataset(Dataset):
    def __init__(self, folder_path, transforms_=None):
        self.transform = transforms.Compose(transforms_)
        self.files = sorted(glob.glob('%s/*.*' % folder_path))

    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)
        return img

    def __len__(self):
        return len(self.files)

In [3]:
img_shape = (3, 100, 100)

cuda = True if torch.cuda.is_available() else False
print(cuda)

True


## Option Settings

In [56]:
n_epochs=5000 #number of epochs of training
batch_size=400 #size of the batches
lr=0.0002 #adam: learning rate
b1=0.5  #"adam: decay of first order momentum of gradient")
b2=0.999 #adam: decay of first order momentum of gradient")
n_cpu=8
latent_dim=100
img_size=200
channels=3
n_critic=5
clip_value=0.01
sample_interval=1000
img_shape = (channels, img_size, img_size)
crop_size = 400
print(img_shape)

(3, 200, 200)


### nn.modules

In [57]:

folder_path = "./zinc100k500px"
transforms_ = [ transforms.Resize(img_size),
                transforms.CenterCrop(img_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(folder_path, transforms_=transforms_),
                        batch_size=batch_size, shuffle=True, num_workers=8)

In [58]:
#img_shape = (3, 100, 100)

#folder_path = "./molecules100"
#transforms_ = [ transforms.Resize(100),
#                transforms.CenterCrop(100),
#                transforms.ToTensor(),
#                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
#dataloader = DataLoader(ImageDataset(folder_path, transforms_=transforms_),
#                        batch_size=100, shuffle=True, num_workers=8)

In [59]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [60]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [61]:
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()


In [62]:
if cuda:
    generator.cuda()
    discriminator.cuda()

TO DO: Determine optimal learning rate

# Actual training now

In [63]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b2, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty



In [64]:
imgs=next(iter(dataloader))

In [65]:
# Sample noise as generator input

imgs
z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
print(z)
print(latent_dim)
print(imgs.shape)
print(imgs)

# Generate a batch of images
fake_imgs = generator(z)


tensor([[ 0.6221,  0.9872,  0.2508,  ..., -0.4239,  0.2535,  0.6143],
        [ 0.4697,  1.0298,  0.1345,  ..., -0.1841, -1.0988,  1.6070],
        [ 1.1106, -0.5274, -0.1055,  ...,  0.2113,  0.5366, -0.5801],
        ...,
        [-1.0739,  0.6992, -0.1014,  ..., -0.3990,  1.0972, -0.4691],
        [-0.7951,  0.4914, -0.8391,  ..., -0.4018, -0.5528,  0.2405],
        [ 0.1755, -0.2522,  0.2104,  ...,  1.2836,  0.4065, -0.2441]],
       device='cuda:0')
100
torch.Size([400, 3, 200, 200])
tensor([[[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          ...,
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000],
          [ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000]],

         [[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.00

In [67]:
batches_done

18605

In [ ]:
# old logger
#from utils import Logger
#logger = Logger(model_name='wGANGP', data_name='mol224')
#num_batches = len(dataloader)
n_epochs=5000
#batches_done = 0
### new logger
#configure("runs/test1",flush_secs=5)

#batch_size=100
#print(latent_dim)
for epoch in range(n_epochs):
    for i, imgs in enumerate(dataloader):
        # Configure input
        real_imgs = Variable(imgs.type(Tensor))
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
        # Generate a batch of images
        fake_imgs = generator(z)
        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
        d_loss.backward()
        optimizer_D.step()
        optimizer_G.zero_grad()
        # Train the generator every n_critic steps
        if i % n_critic == 0:
            # -----------------
            #  Train Generator
            # -----------------
            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)
            g_loss.backward()
            optimizer_G.step()
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

            #logger.log(d_loss, g_loss, epoch, batches_done, num_batches)
            if batches_done % sample_interval/10 ==0:
                log_value('g_loss', g_loss, batches_done)
                log_value('d_loss', d_loss, batches_done)
            
            if batches_done % sample_interval == 0:
                save_image(fake_imgs.data[:25], "molpics500px/%d_c.png" % batches_done, nrow=5, normalize=True)

            batches_done += n_critic

[Epoch 0/5000] [Batch 0/250] [D loss: 3.139673] [G loss: -85.083992]
[Epoch 0/5000] [Batch 5/250] [D loss: 3.247232] [G loss: -92.993080]
[Epoch 0/5000] [Batch 10/250] [D loss: 3.081501] [G loss: -97.527145]
[Epoch 0/5000] [Batch 15/250] [D loss: 3.195412] [G loss: -99.232735]
[Epoch 0/5000] [Batch 20/250] [D loss: 3.142118] [G loss: -97.823044]
[Epoch 0/5000] [Batch 25/250] [D loss: 3.164852] [G loss: -93.660904]
[Epoch 0/5000] [Batch 30/250] [D loss: 3.343189] [G loss: -87.311668]
[Epoch 0/5000] [Batch 35/250] [D loss: 3.375936] [G loss: -78.808334]
[Epoch 0/5000] [Batch 40/250] [D loss: 3.454063] [G loss: -68.854393]
[Epoch 0/5000] [Batch 45/250] [D loss: 3.574682] [G loss: -57.891117]
[Epoch 0/5000] [Batch 50/250] [D loss: 3.651820] [G loss: -46.104927]
[Epoch 0/5000] [Batch 55/250] [D loss: 3.723125] [G loss: -33.906998]
[Epoch 0/5000] [Batch 60/250] [D loss: 3.914867] [G loss: -22.018301]
[Epoch 0/5000] [Batch 65/250] [D loss: 4.045956] [G loss: -10.815705]
[Epoch 0/5000] [Batch 

In [22]:

#tensorboard
!python -m tensorboard.main --logdir runs
#tensorboard --help
#tensorboard --logdir=runs

#logger.d_loss.data.cpu().numpy()
#logger.display_status(epoch, n_epochs, batches_done, num_batches, d_loss, g_loss, d_pred_real, d_pred_fake)

TensorBoard 1.10.0 at http://minilearn:6006 (Press CTRL+C to quit)
W0913 15:35:36.738302 Thread-1 application.py:276] path /[[_dataImageSrc]] not found, sending 404
W0913 15:35:36.738301 139841509275392 application.py:276] path /[[_dataImageSrc]] not found, sending 404
W0913 15:35:36.786560 Thread-1 application.py:276] path /[[_imageURL]] not found, sending 404
W0913 15:35:36.786560 139841509275392 application.py:276] path /[[_imageURL]] not found, sending 404
^C


### Save the models

In [ ]:
os.makedirs("/home/jgmeyer2/vangan/gans/models",exist_ok=True)
PATH = "/home/jgmeyer2/vangan/gans/models/g5k.model"
modelid="5k"


state_g = {
    'epoch': epoch,
    'state_dict': generator.state_dict(),
    'optimizer': optimizer_G.state_dict()
    }
torch.save(state_g, PATH+"g"+modelid+".model")

state_d = {
    'epoch': epoch,
    'state_dict': discriminator.state_dict(),
    'optimizer': optimizer_D.state_dict()
    }
torch.save(state_d, PATH+"d"+modelid+".model")
print("saved models @")
print(epoch)


#def save_model(net, optim, ckpt_fname):
#    state_dict = net.module.state_dict()
#    for key in state_dict.keys():
#        state_dict[key] = state_dict[key].cpu()
#        torch.save({
#            'epoch': epoch,                                                                                                                                                                                     
#            'state_dict': state_dict,                                                                                                                                                                                
#            'optimizer': optim},                                                                                                                                                                                     
#            ckpt_fname)

# Load models

In [ ]:



model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])

model.load_state_dict(state['state_dict'])
optimizer.load_state_dict(state['optimizer'])

state = torch.load(filepath)

